In [1]:
# import pandas as pd

# csv_path = 'test.csv'

# dictionary = {'a':['123','234','345'], 'b':['567','678','789']}
# df = pd.DataFrame.from_dict(dictionary, orient='columns')
# print(df)
# df.to_csv(csv_path)


# Import Necessary Libraries

In [2]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchtext
import math
from PIL import Image
import math
import torch.nn as nn
from torch.nn import functional as F
from mingpt.utils import CfgNode as CN
import time
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore") 

# Global Variables

In [3]:
csv_path = './annotations_t.csv'
image_path = './archive/train_t'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 8
epochs = 110
n_embed = 100
glove = torchtext.vocab.GloVe(name="6B",dim=n_embed) # embedding size = 100  
torch.manual_seed(42)
np.random.seed(42)

# Prepare Dataset

In [4]:
class Custom_Train_Dataset(Dataset):
    def __init__(self, image_folder_path, vocab, csv_path, transformer):
        self.image_folder_path = image_folder_path
        annotations = pd.read_csv(csv_path)
        self.captions = np.array(annotations['captions'])
        self.image_names = np.array(annotations['file_directory'])    
        self.transformer = transformer
        self.vocab = vocab
        
    def __len__(self):
        return len(self.captions)

    
    def __getitem__(self, idx):
        image_full_name = self.image_folder_path + '/' + self.image_names[idx]
        x = Image.open(image_full_name)
        if self.transformer is not None:
            x = self.transformer(x)
#             x = x.permute(1,2,0)
        ground_truth_cap = self.captions[idx]
        V = len(self.vocab.vectors)
        L = ground_truth_cap.split()
        tokenized_caption = torch.tensor([self.vocab.stoi.get(w.lower(), V-1) for w in L])  # Use the last word in the vocab as the "out-of-vocabulary" token
        if V-1 in tokenized_caption:
            print('Wrong Labelling')
#         print(image_full_name, ground_truth_cap)        
        return x, tokenized_caption
    
    def get_vocab_size(self):
        return len(self.vocab.vectors)  
    
    def get_block_size(self):
        all_captions_len = []
        for i in range(len(self.captions)):
            all_captions_len.append(len(self.captions[i].split()))
        return max(all_captions_len)+1
    
custom_transformer = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.RandomHorizontalFlip(p=0.3),
    torchvision.transforms.RandomRotation(degrees=10),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

def lm_collate_fn(batch, device):
    x = [item[0] for item in batch]  
    y_input = [item[1][:-1] for item in batch]  
    y_label = [item[1] for item in batch]
    maxlen_input = max([len(s) for s in y_input])
    maxlen_label = max([len(s) for s in y_label])
    padding_value = glove.stoi.get('unk')
    #x from first word to the second last word, y from second word to the last word
    input_cap, label_cap = [], []
    for sy_i, sy_l in zip(y_input, y_label):
        input_cap.append(torch.cat([sy_i, torch.ones(maxlen_input - len(sy_i))*padding_value]))
        label_cap.append(torch.cat([sy_l, torch.ones(maxlen_label - len(sy_l))*padding_value]))
    return torch.stack(x).long().to(device), torch.stack(input_cap).long().to(device), torch.stack(label_cap).long().to(device)
  
train_dataset = Custom_Train_Dataset(image_path, glove, csv_path, custom_transformer)
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = lambda batch: lm_collate_fn(batch, device))

vocab_size = train_dataset.get_vocab_size()
block_size = train_dataset.get_block_size()


# Model

In [5]:
#model.py
# -----------------------------------------------------------------------------

class NewGELU(nn.Module):

    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.ModuleDict(dict(
            c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd),
            c_proj  = nn.Linear(4 * config.n_embd, config.n_embd),
            act     = NewGELU(),
            dropout = nn.Dropout(config.resid_pdrop),
        ))
        m = self.mlp
        self.mlpf = lambda x: m.dropout(m.c_proj(m.act(m.c_fc(x)))) # MLP forward

    def forward(self, x):
        #x.shape = (batch, sent len, embedding)
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlpf(self.ln_2(x))
        return x

class GPT(nn.Module):
    """ GPT Language Model """

    @staticmethod
    def get_default_config():
        C = CN()
        # either model_type or (n_layer, n_head, n_embd) must be given in the config
        C.model_type = 'gpt'
        C.n_layer = None
        C.n_head = None
        C.n_embd =  None
        # these options must be filled in externally
        C.vocab_size = None
        C.block_size = None
        # dropout hyperparameters
        C.embd_pdrop = 0.1
        C.resid_pdrop = 0.1
        C.attn_pdrop = 0.1
        return C

    def __init__(self, config, vocab):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.block_size = config.block_size

        type_given = config.model_type is not None
        params_given = all([config.n_layer is not None, config.n_head is not None, config.n_embd is not None])
        assert type_given ^ params_given # exactly one of these (XOR)
        if type_given:
            # translate from model_type to detailed configuration
            config.merge_from_dict({
                # names follow the huggingface naming conventions
                # GPT-1
                'openai-gpt':   dict(n_layer=12, n_head=12, n_embd=768),  # 117M params
                # GPT-2 configs
                'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
                'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
                'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
                'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
                # Gophers
                'gopher-44m':   dict(n_layer=8, n_head=16, n_embd=512),
                # (there are a number more...)
                # I made these tiny models up
                'gpt-mini':     dict(n_layer=5, n_head=5, n_embd=n_embed),
                'gpt-micro':    dict(n_layer=4, n_head=4, n_embd=128),
                'gpt-nano':     dict(n_layer=3, n_head=3, n_embd=48),
            }[config.model_type])

        #wte is embedding for words
        #wpe is embedding for positions
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding.from_pretrained(vocab.vectors, freeze = True),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.embd_pdrop),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # init all weights, and apply a special scaled init to the residual projections, per GPT-2 paper
        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters (note we don't count the decoder parameters in lm_head)
        n_params = sum(p.numel() for p in self.transformer.parameters())
        print("number of parameters: %.2fM" % (n_params/1e6,))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    def configure_optimizers(self, train_config, cnn_model_params):
        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name
                # random note: because named_modules and named_parameters are recursive
                # we will see the same tensors p many many times. but doing it this way
                # allows us to know which parent module any tensor p belongs to...
                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )
        if cnn_model_params is not None:
            # create the pytorch optimizer object
            optim_groups = [
                {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
                {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
                {"params": cnn_model_params,'lr': 1e-5}
            ]
            
            n_parameters_transformer = sum(p.numel()
               for p in self.parameters() if p.requires_grad)
            n_parameters_cnn = sum(p.numel()
                           for p in cnn_model_params if p.requires_grad)
            print(f"Number of trainable params: {n_parameters_transformer + n_parameters_cnn}")
        else:
            # create the pytorch optimizer object
            optim_groups = [
                {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
                {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0}
            ]  
        
            n_parameters_transformer = sum(p.numel()
                           for p in self.parameters() if p.requires_grad)
            print(f"Number of trainable params: {n_parameters_transformer}")
        
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer

    def forward(self, image_embed, idx=None, targets=None, finetune_classify=False):

        device = image_embed.device
  
        if idx is not None:
            b, t = idx.size()
            assert t <= self.block_size, f"Cannot forwarnd sequence of length {t}, block size is only {self.block_size}"
            pos = torch.arange(0, t+1, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)

            # forward the GPT model itself
            tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
            tok_emb = torch.cat((image_embed.unsqueeze(1), tok_emb), 1)
          
        else:
            pos = torch.arange(0, 1, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)
            tok_emb = image_embed.unsqueeze(1)   
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
        
        assert tok_emb[0].shape == pos_emb[0].shape, f"wrong token or position embedding"
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        #x.shape = (batch, sentence len, embedding)
        if not finetune_classify:
            # LM forward procedure
            logits = self.lm_head(x)
        else:
            # Finetune classify procedure
            print('error')
            return
            
        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        return logits, loss


In [6]:
# set up model configurations
model_config = GPT.get_default_config()
model_config.model_type = 'gpt-mini'
model_config.vocab_size = vocab_size
#block_size is a max sentence length in dataset
model_config.block_size = block_size
model = GPT(model_config, glove)

number of parameters: 40.61M


# Trainer

In [ ]:
class Trainer:

    @staticmethod
    def get_default_config():
        C = CN()
        # device to train on
        C.device = 'auto'
        # dataloder parameters
        C.num_workers = 4
        # optimizer parameters
        C.max_iters = None
        C.batch_size = 64
        C.learning_rate = 3e-4
        C.betas = (0.9, 0.95)
        C.weight_decay = 0.1 # only applied on matmul weights
        C.grad_norm_clip = 1.0
        return C

    def __init__(self, config, model, train_loader, epochs, downstream_finetune = False):
        self.config = config
        self.model = model
        self.optimizer = None
        self.train_loader = train_loader
        self.epochs = epochs
        self.callbacks = defaultdict(list)
        self.downstream_finetune = False
        # determine the device we'll train on
        if config.device == 'auto':
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = config.device
        self.model = self.model.to(self.device)
        print("running on device", self.device)

        # variables that will be assigned to trainer class later for logging and etc
        self.iter_time = 0.0
        self.iter_dt = 0.0

    def add_callback(self, onevent: str, callback):
        self.callbacks[onevent].append(callback)

    def set_callback(self, onevent: str, callback):
        self.callbacks[onevent] = [callback]

    def trigger_callbacks(self, onevent: str):
        for callback in self.callbacks.get(onevent, []):
            callback(self)

    def run(self):
        model, config = self.model, self.config
        cnn_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True,)
        for param in cnn_model.parameters():
            param.requires_grad = False
        
        cnn_model.classifier = nn.Sequential(nn.Linear(1280, 512),
                                            nn.ReLU(),
                                            nn.Dropout(0.2),
                                            nn.Linear(512, n_embed))
        cnn_model = cnn_model.to(device)
        # setup the optimizer
        self.optimizer = model.configure_optimizers(config, cnn_model.parameters())
        train_loss_set = []
        # setup the dataloader     
        for epoch in range(self.epochs):
            print('Epoch ' + str(epoch) + ':')
            model.train()
            cnn_model.train()
            train_loss_epoch = []
            
            #train model and calculate training accuracy
            for x, y_input, y_label in train_loader:
                # forward the model
                x = x.type(torch.FloatTensor).to(device)
                image_embed = cnn_model(x)
                logits, self.loss = model(image_embed.to(self.device), y_input, y_label, self.downstream_finetune)
                train_loss_epoch.append(self.loss.detach().cpu().item())
                
                # backprop and update the parameters
                model.zero_grad(set_to_none=True)
                self.loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                self.optimizer.step()
                del x, y_input, y_label

            print('Training Loss is : {}'.format(sum(train_loss_epoch)/len(train_loss_epoch)))
            train_loss_set.append(sum(train_loss_epoch)/len(train_loss_epoch))
        
        torch.save(model.state_dict(), 'transformer.pt')
        torch.save(cnn_model.state_dict(), 'cnn.pt')
        
        return train_loss_set

In [ ]:
#start training
train_config = Trainer.get_default_config()
train_config.learning_rate = 8e-5 # the model we're using is so small that we can go a bit faster
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_loader, epochs, downstream_finetune = False)

# Train!
train_loss_set = trainer.run()

# Load Model and Generate Words

In [7]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

cnn_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True,)
cnn_model.classifier = nn.Sequential(nn.Linear(1280, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, n_embed))

model.load_state_dict(torch.load('transformer.pt'))
model.eval()
model.to(device)

cnn_model.load_state_dict(torch.load('cnn.pt'))
cnn_model.eval()
cnn_model.to(device)
max_new_tokens = block_size-1
image_path = 'test_image_2.jpg'

valid_transformer = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

Using cache found in C:\Users\zixua/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\zixua\anaconda3\envs\test0\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\zixua\anaconda3\envs\test0\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
def generate(image_path, device, max_new_tokens, temperature=1.0, top_p=None, top_k=None):
    if top_p is not None and top_k is not None:
        print('Only one sampling approach is allowed')
        return
    if top_p is None and top_k is None:
        print('You must select a sampling approach')
        return
    
    generation_prob = []
    generation_word = []
    idx = None
    image_file = Image.open(image_path)
    image_tensor = valid_transformer(image_file).unsqueeze(0).to(device)
    image_embedding = cnn_model(image_tensor)
    for _ in range(max_new_tokens):
        logits, _ = model(image_embedding, idx)
        # pluck the logits at the final step and scale by desired temperature
        logits = logits[:, -1, :] / temperature
        probs = F.softmax(logits, dim=-1)
        if top_k is not None:
            idx_next_prob, idx_next = torch.topk(probs, k=top_k, dim=-1)
            
        if top_p is not None:
            top_k = 1
            idx_next_prob, idx_next = torch.topk(probs, k=top_k, dim=-1)
            while idx_next_prob.detach().cpu().sum().item() < top_p:
                top_k += 1
                idx_next_prob, idx_next = torch.topk(probs, k=top_k, dim=-1)
                
        idx_next_prob, idx_next = idx_next_prob[0].detach().cpu(), idx_next[0].detach().cpu()
        sum_prob = idx_next_prob.sum().item()
        idx_next_prob = [prob_i/sum_prob for prob_i in idx_next_prob.tolist()]
        idx_next_prob[-1] = 1 - sum(idx_next_prob[:-1])

        select_token = np.random.choice(idx_next.tolist(), 1, p=idx_next_prob)  
        for token, token_prob in zip(idx_next, idx_next_prob):
            if token == select_token[0]:
                generation_prob.append(token_prob)    

        generation_word.append(glove.itos[select_token[0]])
        test_prob, test_next = torch.topk(probs, k=2, dim=-1)
        print(test_prob)
        print(glove.itos[test_next[0, 0].detach().cpu().item()], glove.itos[test_next[0, 1].detach().cpu().item()])            
            
            
        idx_next = torch.tensor(select_token).unsqueeze(0).to(device)
        if idx is None:
            # append sampled index to the running sequence and continue
            idx = idx_next
        else:
            idx = torch.cat((idx, idx_next), dim=1)
        
        if generation_word[-1] == '.':
            break

    return generation_word, generation_prob

generation_word, generation_prob = generate(image_path, device, max_new_tokens, temperature=0.01, top_p=None, top_k=1)
print(generation_word)

tensor([[0.7190, 0.2810]], device='cuda:0', grad_fn=<TopkBackward0>)
he i
tensor([[1., 0.]], device='cuda:0', grad_fn=<TopkBackward0>)
is the
tensor([[1., 0.]], device='cuda:0', grad_fn=<TopkBackward0>)
a the
tensor([[1., 0.]], device='cuda:0', grad_fn=<TopkBackward0>)
dog the
tensor([[1., 0.]], device='cuda:0', grad_fn=<TopkBackward0>)
. the
['he', 'is', 'a', 'dog', '.']


# Compute Bleu Score

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    'he is a not dog .'.split(),
    'he is a good dog .'.split(),
    'he is the best dog .'.split()
]
print(reference)
print('BLEU score -> {}'.format(sentence_bleu(reference, generation_word, weights=(0.7,0.2,0.1))))
#https://towardsdatascience.com/foundations-of-nlp-explained-bleu-score-and-wer-metrics-1a5ba06d812b